In [69]:
import pandas as pd
import os
import numpy as np

In [2]:
DATA_FOLDER="data_final/"

# Reading Data 

### Прочитаем данные

In [16]:
transactions    = pd.read_csv(os.path.join(DATA_FOLDER, 'sales_train.csv.gz'))
items           = pd.read_csv(os.path.join(DATA_FOLDER, 'items.csv'))
item_categories = pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories.csv'))
shops           = pd.read_csv(os.path.join(DATA_FOLDER, 'shops.csv'))
smpsb_df = pd.read_csv(os.path.join(
    DATA_FOLDER,
    "sample_submission.csv.gz"),
    header=0, sep=',', quotechar='"')

In [17]:
test_df = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv.gz'))

In [18]:
transactions.date=pd.to_datetime(transactions.date,format='%d.%m.%Y')
transactions['day']= transactions.date.dt.day
transactions['month'] = transactions.date.dt.month
transactions['year'] = transactions.date.dt.year

## Preprocessing data 

### add categories

In [29]:
l = list(item_categories.item_category_name)
l_cat = l

for ind in range(0,8):
    l_cat[ind] = 'Access'

for ind in range(10,18):
    l_cat[ind] = 'Consoles'

for ind in range(18,25):
    l_cat[ind] = 'Consoles Games'

for ind in range(26,28):
    l_cat[ind] = 'Phone games'

for ind in range(28,32):
    l_cat[ind] = 'CD games'

for ind in range(32,37):
    l_cat[ind] = 'Card'

for ind in range(37,43):
    l_cat[ind] = 'Movie'

for ind in range(43,55):
    l_cat[ind] = 'Books'

for ind in range(55,61):
    l_cat[ind] = 'Music'

for ind in range(61,73):
    l_cat[ind] = 'Gifts'

for ind in range(73,79):
    l_cat[ind] = 'Soft'


item_categories['cat'] = l_cat
item_categories.head()

,item_category_name,item_category_id,cat
0,PC - Гарнитуры/Наушники,0,Access
1,Аксессуары - PS2,1,Access
2,Аксессуары - PS3,2,Access
3,Аксессуары - PS4,3,Access
4,Аксессуары - PSP,4,Access


### add location

In [43]:
from sklearn.preprocessing import LabelEncoder

In [44]:
shops.loc[shops.shop_name == 'Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'
shops['city'] = shops['shop_name'].str.split(' ').map(lambda x: x[0])
shops.loc[shops.city == '!Якутск', 'city'] = 'Якутск'
shops['city_code'] = LabelEncoder().fit_transform(shops['city'])
shops = shops[['shop_id','city_code']]

## Embedding 

In [55]:
import gensim.models
import fasttext

# items.item_name.to_csv("data.txt",encoding="utf-8")
model = fasttext.skipgram('data.txt', 'model1',dim=100,minn=2,maxn=5)

model1 = gensim.models.FastText.load_fasttext_format("model1.bin")



In [73]:
model1.wv.vector_size

100

In [75]:
def avg_feature_vector(sentence, model):
    words = sentence.split()
    feature_vec = np.zeros((model.vector_size, ), dtype='float32')
    n_words = 0
    for word in words:
        feature_vec = np.add(feature_vec, model[word])
    feature_vec = np.divide(feature_vec, len(words))
    return feature_vec

In [112]:
items_name = items['item_name'].values

In [123]:
A=np.array(list(map(lambda x: avg_feature_vector(x,model1.wv),items_name)))

##  EDA